## Importing dataset containing game date, game ID, team IDs, season, and game results

In [1]:
# Importing initial data
import pandas as pd
pd.options.display.max_rows = 1235
df = pd.read_csv('games.csv')

# Initital preprocessing of data
df = df.drop(columns=['GAME_STATUS_TEXT','TEAM_ID_home','FG_PCT_home','FT_PCT_home','FG3_PCT_home','AST_home','REB_home','TEAM_ID_away','FG_PCT_away','FT_PCT_away','FG3_PCT_away','AST_away','REB_away'])
## rename VISITOR_TEAM_ID to AWAY_TEAM_ID
df = df.rename(columns={'VISITOR_TEAM_ID':'AWAY_TEAM_ID'})
## create new column equal to margin of victory/defeat for home team
df['MARGIN'] = (df['PTS_home'] - df['PTS_away'])
df = df.drop(columns=['PTS_home','PTS_away'])
df = df.dropna()

# Data type conversions (GAME_DATE_EST -> date, TEAM_ID -> categorical)
df['GAME_DATE_EST'] = pd.to_datetime(df.GAME_DATE_EST, infer_datetime_format=True)
df['HOME_TEAM_ID'] = df['HOME_TEAM_ID'].astype('category')
df['AWAY_TEAM_ID'] = df['AWAY_TEAM_ID'].astype('category')

In [2]:
# Eliminating 2003 & 2004 seasons (incomplete team stats data from previous year for all 30 teams)
df = df[~(df['SEASON'] == 2003) & ~(df['SEASON'] == 2004)]

# Eliminating 2019 & 2020 seasons (incomplete)
df = df[~(df['SEASON'] == 2019) & ~(df['SEASON'] == 2020)]

# Eliminate preseason and playoff games
s2005 = df[df['SEASON'] == 2005]
s2005 = s2005[~(s2005['GAME_DATE_EST'] < '2005-11-01') & ~(s2005['GAME_DATE_EST'] > '2006-04-19')]

s2006 = df[df['SEASON'] == 2006]
s2006 = s2006[~(s2006['GAME_DATE_EST'] < '2006-10-31') & ~(s2006['GAME_DATE_EST'] > '2007-04-18')]

s2007 = df[df['SEASON'] == 2007]
s2007 = s2007[~(s2007['GAME_DATE_EST'] < '2007-10-30') & ~(s2007['GAME_DATE_EST'] > '2008-04-16')]

s2008 = df[df['SEASON'] == 2008]
s2008 = s2008[~(s2008['GAME_DATE_EST'] < '2008-10-28') & ~(s2008['GAME_DATE_EST'] > '2009-04-16')]

s2009 = df[df['SEASON'] == 2009]
s2009 = s2009[~(s2009['GAME_DATE_EST'] < '2009-10-27') & ~(s2009['GAME_DATE_EST'] > '2010-04-14')]

s2010 = df[df['SEASON'] == 2010]
s2010 = s2010[~(s2010['GAME_DATE_EST'] < '2010-10-26') & ~(s2010['GAME_DATE_EST'] > '2011-04-13')]

s2011 = df[df['SEASON'] == 2011]
s2011 = s2011[~(s2011['GAME_DATE_EST'] < '2011-12-25') & ~(s2011['GAME_DATE_EST'] > '2012-04-26')]

s2012 = df[df['SEASON'] == 2012]
s2012 = s2012[~(s2012['GAME_DATE_EST'] < '2012-10-30') & ~(s2012['GAME_DATE_EST'] > '2013-04-17')]

s2013 = df[df['SEASON'] == 2013]
s2013 = s2013[~(s2013['GAME_DATE_EST'] < '2013-10-29') & ~(s2013['GAME_DATE_EST'] > '2014-04-16')]

s2014 = df[df['SEASON'] == 2014]
s2014 = s2014[~(s2014['GAME_DATE_EST'] < '2014-10-28') & ~(s2014['GAME_DATE_EST'] > '2015-04-15')]

s2015 = df[df['SEASON'] == 2015]
s2015 = s2015[~(s2015['GAME_DATE_EST'] < '2015-10-27') & ~(s2015['GAME_DATE_EST'] > '2016-04-13')]

s2016 = df[df['SEASON'] == 2016]
s2016 = s2016[~(s2016['GAME_DATE_EST'] < '2016-10-25') & ~(s2016['GAME_DATE_EST'] > '2017-04-12')]

s2017 = df[df['SEASON'] == 2017]
s2017 = s2017[~(s2017['GAME_DATE_EST'] < '2017-10-17') & ~(s2017['GAME_DATE_EST'] > '2018-04-11')]

s2018 = df[df['SEASON'] == 2018]
s2018 = s2018[~(s2018['GAME_DATE_EST'] < '2018-10-16') & ~(s2018['GAME_DATE_EST'] > '2019-04-10')]

# Put individual years back into df after removing preseason & playoffs
df = pd.concat([s2018,s2017,s2016,s2015,s2014,s2013,s2012,s2011,s2010,s2009,s2008,s2007,s2006,s2005])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16979 entries, 16603 to 13297
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   GAME_DATE_EST   16979 non-null  datetime64[ns]
 1   GAME_ID         16979 non-null  int64         
 2   HOME_TEAM_ID    16979 non-null  category      
 3   AWAY_TEAM_ID    16979 non-null  category      
 4   SEASON          16979 non-null  int64         
 5   HOME_TEAM_WINS  16979 non-null  int64         
 6   MARGIN          16979 non-null  float64       
dtypes: category(2), datetime64[ns](1), float64(1), int64(3)
memory usage: 832.0 KB


## Importing rankings dataset with winning percentages (current season)

In [3]:
# Importing ranking data and eliminating unnecessary columns
ranking = pd.read_csv('ranking.csv')
ranking = ranking[['TEAM_ID','STANDINGSDATE','W_PCT']]

# Data type conversions (GAME_DATE_EST -> date, TEAM_ID -> categorical)
ranking['STANDINGSDATE'] = pd.to_datetime(ranking.STANDINGSDATE, infer_datetime_format=True)
ranking['TEAM_ID'] = ranking['TEAM_ID'].astype('category')

## Adding winning percentages for each team (current season)

In [4]:
# Join df & ranking (add column of W_PCT for home team)
df = pd.merge(df, ranking,  how='left', left_on=['GAME_DATE_EST','HOME_TEAM_ID'], right_on = ['STANDINGSDATE','TEAM_ID'])

df = df.rename(columns={'W_PCT':'HOME_W%'})
df = df.drop(columns=['STANDINGSDATE','TEAM_ID'])

In [5]:
# Join df & ranking (add column of W_PCT for away team)
df = pd.merge(df, ranking,  how='left', left_on=['GAME_DATE_EST','AWAY_TEAM_ID'], right_on = ['STANDINGSDATE','TEAM_ID'])

df = df.rename(columns={'W_PCT':'AWAY_W%'})
df = df.drop(columns=['STANDINGSDATE','TEAM_ID'])

## Finalize and export base dataset

In [6]:
# Rearrange columns as needed
cols = ['GAME_DATE_EST','GAME_ID','SEASON','HOME_TEAM_ID','HOME_W%','AWAY_TEAM_ID','AWAY_W%','HOME_TEAM_WINS','MARGIN']
df = df[cols]

df.to_csv('baseGames.csv')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16979 entries, 0 to 16978
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   GAME_DATE_EST   16979 non-null  datetime64[ns]
 1   GAME_ID         16979 non-null  int64         
 2   SEASON          16979 non-null  int64         
 3   HOME_TEAM_ID    16979 non-null  category      
 4   HOME_W%         16979 non-null  float64       
 5   AWAY_TEAM_ID    16979 non-null  category      
 6   AWAY_W%         16979 non-null  float64       
 7   HOME_TEAM_WINS  16979 non-null  int64         
 8   MARGIN          16979 non-null  float64       
dtypes: category(2), datetime64[ns](1), float64(3), int64(3)
memory usage: 1.1 MB


# Counting a unique value
len(pd.unique(df['GAME_DATE_EST']))

# Export Pandas Profiling Report
pip install pandas-profiling
from pandas_profiling import ProfileReport
pandasProfileReport = ProfileReport(df)
pandasProfileReport.to_file(output_file='pandasProfile.html')

# Plot correlation matrix
import seaborn as sn
import matplotlib.pyplot as plt
corrMatrix = df.corr()
sn.heatmap(corrMatrix, annot=True)
plt.show()